# 程序猿王大锤的指数基金投资机器人

上回说到投资组合优化和指数投资（超链接），并给出了简单的投资建议，但是这种朴素的方法并不能适应市场的需求。我们需要在市场中实际检验它的效果。一种合理的方法是使用历史情景去进行模拟操作，这种方法就叫做回测（backtest）。

与简单的使用历史数据不同，进行回测实验需要考虑现实生活中可能发生的各种情况，每一个疏忽都可能会让算法失效。某种意义上来说，构建回测策略相当于构建一个投资机器人，它通过读取历史数据作为输入，通过买卖资产与现实世界进行交互，以收益最优化作为其人生目标。

这个算法有很成熟的工业实践，但是王大锤决定自己造人。毛爷爷告诉我们，做人要学会认识世界，改造世界。机器人也要如此，它必须具有一定的世界观和方法论。简单来说，要让它对这个世界心里有点x数，这个通过我们的投资组合优化理论实现，其次要让它可以通过自己的想法进行操作，这个就是王大锤的核心算法了。

通过回忆本科时候的单片机实验，王大锤决定搞一个状态机出来做这件事情。做这件事情最简单的状态机有三个状态，买、卖、等，实现出来大概就长下面这个模样（感谢优矿平台 https://uqer.io/labs/ 为本机器人提供除脑子之外的其他部分）：

In [ ]:
# 强行让python2变得像python3
from __future__ import print_function, unicode_literals, division, generators, with_statement
import pandas as pd
import datetime as dt
import numpy as np
import math
from matplotlib import pyplot as plt
import scipy.optimize as sco
import operator

pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 1000)

In [ ]:
df_funds = DataAPI.FundGet()
df_index = df_funds[df_funds.indexFund.isnull() != True][df_funds.isClass == 1][df_funds.exchangeCd.isnull() == True ]

In [ ]:
df_index

In [ ]:
def efficient_frontier(returns):
    '''
    '''
    n = returns.shape[1]
    noa = returns.shape[0]
    
    N = 5
    qs = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    Sigma = np.cov(returns)
    RT = np.mean(returns,axis=1)
    
    cons = ({'type':'eq','fun':lambda x:np.sum(x)-1})
    bnds = tuple((0,1) for x in range(noa))
    
    rets = []
    risks = []
    weights = []
    
    for q in qs:
        def markowitz_loss(weights):
            wT = weights.flatten()
            w = wT.T
            loss = wT.dot(Sigma).dot(w) - q * RT.dot(w)
            return loss
    
        res = sco.minimize(markowitz_loss, noa*[1./noa,], method='SLSQP', bounds=bnds, constraints=cons)
        rets.append(RT.dot(res.x.T))
        risks.append(math.sqrt(res.x.T.dot(Sigma).dot(res.x)))
        weights.append(res.x)
        
    rets = np.array(rets)
    risks = np.array(risks)
    return rets,risks,weights

In [ ]:
start = '2016-09-01'                       # 回测起始时间
end = '2017-09-01'                         # 回测结束时间
universe = df_index.secID.tolist()         # 证券池，支持股票和基金、期货
benchmark = 'HS300'                        # 策略参考基准
freq = 'd'                                 # 'd'表示使用日频率回测，'m'表示使用分钟频率回测
refresh_rate = 1                           # 执行handle_data的时间间隔

step = 30
window = 50
capital_base = 100000
invest_ratio = 0.95

STATE_RELOCATION = 0
STATE_PURCHASE = 1
STATE_HALT = -1

accounts = {
    'fantasy_account': AccountConfig(account_type='otc_fund', capital_base=capital_base)
}

max_history_window = window

def initialize(context):                   # 初始化策略运行环境
    context.tick = 0
    context.my_universe = None
    context.state = 0
    
    context.weights = None
    context.target_positions = None

def handle_data(context):                  # 核心策略逻辑
    # if (context.current_date - dt.datetime.strptime(start,'%Y-%m-%d')).days < window:
    #     return
    
    account = context.get_account('fantasy_account')
    positions = account.get_positions()
    
    capital = account.cash
    for k,v in positions.items():
        capital += v.value
        
    # print(context.current_date, context.tick, context.state, account.cash, positions)

    if context.tick == 0:
        context.state = STATE_RELOCATION
        
    elif context.tick == step:
        context.tick = -1
    
    else:
        pass
    
    if context.state == STATE_RELOCATION: # Relocation
        universe = context.get_universe('otc_fund',exclude_halt=True)
        df_today = context.history(symbol=universe,attribute='nav',style='ast')['nav']
        context.my_universe = df_today.columns.tolist()
        
        df = context.history(symbol=context.my_universe,attribute='nav',time_range=window,style='ast')['nav']
        df.fillna(method='ffill', inplace=True)
        df.fillna(method='bfill', inplace=True)
        df.fillna(value=1.0, inplace=True)
        
        df_returns = df.pct_change().dropna()
        df_returns.values[np.abs(df_returns.values) > 0.1] = 0
        
        rets,risks,weights = efficient_frontier(df_returns.values.T)
        
        context.weights = np.round(weights[-1], decimals=2)
        context.target_positions = np.zeros_like(context.weights)
        
        for i in range(context.weights.shape[0]):
            target = capital * context.weights[i] * invest_ratio
            context.target_positions[i] = target
            
            if positions.has_key(universe[i]):
                position = positions[universe[i]].value
                if position - target > 10:
                    account.redeem(universe[i], (position - target) / position * positions[universe[i]].amount - 0.01)
                    print('sell', universe[i], (position - target) / position * positions[universe[i]].amount - 0.01)
                    
        context.state = STATE_PURCHASE
                       
    elif context.state == STATE_PURCHASE: # Purchase
        universe = context.my_universe
        df_today = context.history(symbol=universe,attribute='nav',style='ast')['nav']
        available_fund = set(df_today.columns)
        
        b_complete = True
        cnt = 0
        for i in range(context.weights.shape[0]):
            target = context.target_positions[i]
            
            position = 0
            if positions.has_key(universe[i]):
                position = positions[universe[i]].value
            if target - position > 10:
                b_complete = False
                amount = target - position
                if amount >= account.cash:
                    amount = account.cash

                if amount > 10:
                    if universe[i] in available_fund and not np.isnan(df_today[universe[i]][0]):
                        # cnt += 1
                        # if cnt > 10:
                        #     break
                        account.purchase(universe[i], amount - 0.01)
                        print('buy', universe[i], amount - 0.01)
                    else:
                        print('%s is not available!' % universe[i])
        if b_complete:
            context.state == STATE_HALT
            
    elif context.state == STATE_HALT:
        pass
        
    context.tick += 1

虽然看起来不错，但是以上方法依然存在很多问题，最主要的问题在于没有止损策略，容易大幅亏损。欲知后事如何，且听下回分解。